<a href="https://colab.research.google.com/github/zihoonkim/practice/blob/main/%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 라이브러리 불러오기
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 단일 레이어 구현 함수
def affine_layer_forward(X, W, b):
    y = np.dot(X, W) + b
    cache = (X, W, b)
    return y, cache

print('✔')

In [11]:
# sigmoid 함수를 구현해 봅니다.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  

print('✔')

# softmax함수를 구현해 봅니다.
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

print('✔')

# 정답 라벨을 One-hot 인코딩하는 함수
def _change_one_hot_label(X, num_category): ## 여기서 x는 정답 array
    T = np.zeros((X.size, num_category))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
        
    return T

✔
✔


In [4]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

print('✔')

✔


In [5]:
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

print('✔')

✔


In [6]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    return W1, b1, W2, b2

print('✔')

✔


In [7]:
def affine_layer_backward(dy, cache):
    X, W, b = cache
    dX = np.dot(dy, W.T)
    dW = np.dot(X.T, dy)
    db = np.sum(dy, axis=0)
    return dX, dW, db

print('✔')

✔


In [8]:
# MNIST 데이터를 로드. 다운로드하지 않았다면 다운로드까지 자동으로 진행됩니다. 
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 모델에 맞게 데이터 가공
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
x_train_reshaped = x_train_norm.reshape(-1, x_train_norm.shape[1]*x_train_norm.shape[2])
x_test_reshaped = x_test_norm.reshape(-1, x_test_norm.shape[1]*x_test_norm.shape[2])

# 테스트를 위해 x_train_reshaped의 앞 5개의 데이터를 가져온다.
X = x_train_reshaped[:5]
print(X.shape)

11501568/11490434 [==============================] - 0s 0us/step
(5, 784)


In [22]:
weight_init_std = 0.1
input_size = 784
hidden_size=50
output_size = 10
Y_digit = y_train[:5]

# 파라미터 초기화
W1 = weight_init_std * np.random.randn(input_size, hidden_size) # 첫번째 가중치를 만들기 위해 std에 784 * 50 차원의 난수를 만들어
b1 = np.zeros(hidden_size)  ## hidden size에 bias를 넣기 위해
W2 = weight_init_std * np.random.randn(hidden_size, output_size) # 두번째 처리를 위해 50 * 10 차원 만들고
b2 = np.zeros(output_size) # 결과값에 bias 넣기 위해 설정해 

# Forward Propagation
a1, cache1 = affine_layer_forward(X, W1, b1)  # 이 과정을 통해  y = np.dot(x, w1) +b1의 식을 만들고
z1 = sigmoid(a1) # 위의 결과값 y 즉 ai을 활성화 함수를 지나기 위해 시그모이드를 지나
a2, cache2 = affine_layer_forward(z1, W2, b2) # 여기서는 입력층이 z1이 되니까 z1으로 위의 과정 다시

# 추론과 오차(Loss) 계산
y_hat = softmax(a2)  # 출력값을 보기 위한 softmax 함수를 만들어 예측값 y_hat을 만들고 
t = _change_one_hot_label(Y_digit, 10)   # 정답 One-hot 인코딩
Loss = cross_entropy_error(y_hat, t)

print(y_hat)
print(t)
print('Loss: ', Loss)
        
dy = (y_hat - t) / X.shape[0]
dz1, dW2, db2 = affine_layer_backward(dy, cache2)
da1 = sigmoid_grad(a1) * dz1
dX, dW1, db1 = affine_layer_backward(da1, cache1)

# 경사하강법을 통한 파라미터 업데이트    
learning_rate = 0.1
print('계산해서 나온 더하거나 뺄 값')
print(dW2[0])
print('오차역전법 수행전')
print(W2[0])
W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
print('오차역전법 수행후')
print(W2[0])

[[0.08060149 0.10585844 0.13861279 0.06849028 0.06659322 0.06420037
  0.12514785 0.12479501 0.06698774 0.1587128 ]
 [0.09007495 0.09798632 0.09987315 0.06888219 0.0818047  0.09882861
  0.11802486 0.12127207 0.09088263 0.13237052]
 [0.09166593 0.09517062 0.13842931 0.06203884 0.06222869 0.07799161
  0.12111158 0.1319341  0.07414651 0.14528282]
 [0.08848303 0.09028792 0.14204249 0.07095549 0.06908566 0.06767796
  0.1062082  0.10615592 0.07406343 0.18503989]
 [0.10270132 0.10071223 0.10753474 0.06498927 0.07315781 0.09521317
  0.10810333 0.10905437 0.07730073 0.16123303]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Loss:  2.431890988888254
계산해서 나온 더하거나 뺄 값
[-0.02819456 -0.085213    0.04613494  0.02427742 -0.04887363 -0.0197574
  0.04104381  0.04207387  0.02751116  0.00099739]
오차역전법 수행전
[-0.0706738  -0.07715706  0.03843812 -0.03977835 -0.05058867 -0.14830001
  0.0116465